In [1]:
import csv

import pandas as pd

import numpy as np

import torch

from torch import nn, optim

from torch.autograd import Variable

In [2]:
df = pd.read_csv('formatweather.csv')
#train_x=df.values[:,1:9]   #shape:(43823, 8)
#train_y=df.values[:,9]     #shape:(43823,)
train_x=np.asarray(df.values[:,1:9], dtype =  np.float32)
train_y=np.asarray(df.values[:,9], dtype =  np.float32)   ##dtype = np.float32

In [3]:
train_x = torch.from_numpy(train_x)  #43823,1,1,8
train_y = torch.from_numpy(train_y)  #43823,1
train_x=train_x.view(-1,1,1,8)
train_y=train_y.view(-1,1)

In [4]:
class Cnn(nn.Module):

    def __init__(self, in_dim, n_class):

        super(Cnn, self).__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(in_dim, 6, (1,2), stride=(1), padding=(0,1)),#6,1,9

            nn.ReLU(True),

            nn.MaxPool2d(1, 1),#6,1,9

            nn.Conv2d(6, 16, (1,3), stride=(1), padding=(0,1)),

            nn.ReLU(True), nn.MaxPool2d(1, 1))#16,1,9
   
            #5*144

        self.fc = nn.Sequential(

            nn.Linear(144, 16), nn.Linear(16, 8), nn.Linear(8, n_class))



    def forward(self, x):

        out = self.conv(x)

        out = out.view(out.size(0), -1)

        out = self.fc(out)
        
        #out = out.view(5)

        return out

In [5]:
model = Cnn(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [6]:
for epoch in range(1000):
    
    out = model(Variable(train_x))
  
    loss = criterion(out, Variable(train_y))

    # running_loss += loss.item() * train_y.size(0)
    if (epoch+1)%10==0:
        print('epoch {}'.format(epoch + 1))

        print('*' * 10)
        print(loss)   

    # 向后传播

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
print(out,train_y)

# 保存模型
torch.save(model.state_dict(), './weathercnn.pth')

epoch 10
**********
tensor(0.1469, grad_fn=<MseLossBackward>)
epoch 20
**********
tensor(0.1327, grad_fn=<MseLossBackward>)
epoch 30
**********
tensor(0.1201, grad_fn=<MseLossBackward>)
epoch 40
**********
tensor(0.1089, grad_fn=<MseLossBackward>)
epoch 50
**********
tensor(0.0990, grad_fn=<MseLossBackward>)
epoch 60
**********
tensor(0.0901, grad_fn=<MseLossBackward>)
epoch 70
**********
tensor(0.0822, grad_fn=<MseLossBackward>)
epoch 80
**********
tensor(0.0751, grad_fn=<MseLossBackward>)
epoch 90
**********
tensor(0.0687, grad_fn=<MseLossBackward>)
epoch 100
**********
tensor(0.0630, grad_fn=<MseLossBackward>)
epoch 110
**********
tensor(0.0578, grad_fn=<MseLossBackward>)
epoch 120
**********
tensor(0.0532, grad_fn=<MseLossBackward>)
epoch 130
**********
tensor(0.0490, grad_fn=<MseLossBackward>)
epoch 140
**********
tensor(0.0452, grad_fn=<MseLossBackward>)
epoch 150
**********
tensor(0.0418, grad_fn=<MseLossBackward>)
epoch 160
**********
tensor(0.0387, grad_fn=<MseLossBackward>)
e

In [ ]:
out

In [ ]:
train_y

In [ ]:
x=np.array([[0.129779 ,0.352941 ,0.245902 ,0.527273 ,0.666667 ,0.002290 ,0.000000 ,0.0],  
        [0.148893 ,0.367647 ,0.245902 ,0.527273 ,0.666667 ,0.003811 ,0.000000 ,0.0],
        [0.159960 ,0.426471 ,0.229508 ,0.545454 ,0.666667 ,0.005332 ,0.000000 ,0.0],
        [0.182093 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.008391 ,0.037037 ,0.0],
        [0.138833 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.009912 ,0.074074 ,0.0]],dtype = np.float32)
y=np.array([0.1 ,0.1 ,0.1 ,0.1 ,0.1])